# Import Important Python Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-sspoy06y because the default path (/home/gcatangui/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


# Import Datasets

In [2]:
driver_test = pd.read_csv('drivers_test.csv')
driver_test2 = pd.read_csv('drivers_test2.csv')
locations = pd.read_csv('locations.csv')
trips_test = pd.read_csv('trips_test.csv')
trips_test2 = pd.read_csv('trips_test2.csv')

In [3]:
print(trips_test.shape)
print(trips_test2.shape)

(4, 9)
(10000, 9)


In [4]:
#Append Datasets
trips = trips_test.append(trips_test2)
#trips['pickup_datetime'] = pd.to_datetime(trips['pickup_datetime'])
trips_csv = trips.to_csv('trips.csv') # Save data to CSV
display(trips)

/tmp/ipykernel_610/2141425324.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trips = trips_test.append(trips_test2)


,trip_id,driver_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_loc_id,dropoff_loc_id,trip_distance,fare_amount
0,1,1,"08:13:00,15-05-2022","08:46:00,15-05-2022",2,1,2,17.600000,412.000000
1,2,2,"14:13:00,31-12-2022","14:46:00,31-12-2022",1,3,4,15.100000,371.000000
2,3,3,"09:13:00,16-08-2022","09:46:00,16-08-2022",3,5,2,8.900000,235.000000
3,4,1,"15:13:00,09-09-2022","15:46:00,09-09-2022",2,6,7,31.200000,716.000000
0,0,116,"20:22:02,01-02-2022","20:41:26,01-02-2022",1,27,17,57.335208,75.834505
...,...,...,...,...,...,...,...,...,...
9995,9995,12,"19:38:54,18-08-2022","19:58:18,18-08-2022",3,43,28,24.905899,55.566187
9996,9996,10,"14:56:10,15-03-2022","15:15:34,15-03-2022",1,13,49,77.739584,88.587240
9997,9997,134,"08:13:38,15-06-2022","08:33:02,15-06-2022",0,14,40,73.393433,85.870896
9998,9998,48,"11:53:35,01-02-2022","12:12:59,01-02-2022",0,31,23,99.555478,102.222174


#### Adding trips in the database

Create a method `add_trips` that accepts a list of trips in the form of dictionaries with the following keys:
  - `driver`: trip driver as a string in `Last name, Given name` format
  - `pickup_datetime`: datetime of pickup as string with format "hh:mm:ss,DD-MM-YYYY"
  - `dropoff_datetime`: datetime of dropoff as string with format "hh:mm:ss,DD-MM-YYYY"
  - `passenger_count`: number of passengers as integer
  - `pickup_loc_name`: zone as a string, (e.g., Pine View, Legazpi Village)
  - `dropoff_loc_name`: zone as a string, (e.g., Pine View, Legazpi Village)
  - `trip_distance`: distance in meters (float)
  - `fare_amount`: amount paid by passenger (float)
  
The method should add each trip to the database. 
It returns a list of the `trip_ids`s of successfully added trips. If a trip is already in the database, skip it and print: `Warning: trip index {i} is already in the database. Skipping...`

If a trip has invalid or incomplete information, skip it and print `Warning: trip index {i} has invalid or incomplete information. Skipping...` instead. The *trip index* is the zero-based index of the trip in the passed list of trips.

In [8]:
def add_trip(self,
             driver,
             pickup_datetime,
             dropoff_datetime,
             passenger_count,
             pickup_loc_name,
             dropoff_loc_name,
             trip_distance,
             fare_amount):
        
        # check the format
        pickup_datetime = pd.to_datetime('hh:mm:ss,DD-MM-YYYY')
        dropoff_datetime = pd.to_datetime('hh:mm:ss,DD-MM-YYYY')
        
        # prepare the driver id, SHOULD BE CASE INSENSITIVE
        drivers = pd.read_csv(f'{self.data_dir}/drivers.csv')
        driver = [x.strip() for x in driver.split(',')]
        driver_id = drivers[(drivers['last_name']==driver[0]) & (drivers['given_name']==driver[1])]
        
        if len(driver_id)!=0:
            driver_id = ['driver_id'].iloc[0]
        else:
            driver_id = drivers['driver_id'].max() + 1
            new_driver = pd.DataFrame({'driver_id': [driver_id], 'last_name': [driver[0]], 'given_name': [driver[1]]})
            drivers = pd.concat([drivers, new_driver], axis=0)
            drivers.to_csv(f'{self.data_dir}/drivers.csv')
        
        # prepare the pickup/dropoff loc id
        pickup_loc_id = locations.loc[(locations['loc_name']==pickup_loc_name), 'location_id'].iloc[0]
        dropoff_loc_id = locations.loc[(locations['loc_name']==dropoff_loc_name), 'location_id'].iloc[0]
        
        #dropoff_loc_name
        dropoff_loc_name = dropoff_loc_name.astype('string')
                    
        #trip_distance
        trip_distance = dropoff_loc_name.astype('float')
        
        #fare_amount
        fare_amount = fare_amount.astype('float')
        
        try:
            trips = pd.read_csv(f'{self.data_dir}/trips.csv')
            trip_id = trips['trip_id'].max() + 1
            
            # check if trip is existing, if not add
            # return trip id/error
            
        except:
            trips = pf.DataFrame({'trip_id': 1,
                                  'driver_id': driver_id,
                                  'pickup_datetime': pickup_datetime,
                                  'dropoff_datetime': dropoff_datetime,
                                'passenger_count', passenger_count,
                                  'pickup_loc_id': pickup_loc_id,
                                  'dropoff_loc_id': dropoff_loc_id,
                                'trip_distance': trip_distance,
                                  'fare_amount': fare_amount})
            
            trips.to_csv(f'{self.data_dir}/trips.csv'
        
            # return trip id

SyntaxError: invalid syntax (836580779.py, line 53)

In [ ]:
def delete_trip(self, trip_id):
        # Load/define trips_csv
        try:
            # filter the data
            # remove from trips_csv
            # save clean dataframe
        except:
             raise SakayDBError

In [ ]:
#Validate if trip is already in the database
    def validate_trip(self, trip_id):
        if self.trip_id.isin(remove_trip) == True:
            print('Trip_id is Trips.csv')
        else: 
            raise TypeError("trip index {i} is already in the database. Skipping...")

#### Deleting a trip in the database

Create a method `delete_trip` that accepts the trip_id to delete then removes it from `trips.csv` . It will raise a `SakayDBError` if the `trip_id` is not found

In [9]:
def delete_trip(trip_id):
        # Load/define trips_csv
        trips = trips.to_csv()
        try:
            # filter the data
            trips_filter = trips['trip_id'] == trips_id
            # remove from trips_csv
                remove_trip = trips.drop(trips_filter)
            # save clean dataframe
            remove_trip.to_csv('trips.csv')
        except:
             raise SakayDBError

IndentationError: unexpected indent (2572381300.py, line 8)

In [ ]:
# Remove Trip from trips.csv
    def remove_trip(self, trip_id):
        # Load/define trips_csv
        trips_csv = set(trips['trip_id'].to_list())
        # remove from trips_csv
        remove_trip = trips_csv.remove(trip_id)
        return remove_trip.to_csv('trips.csv')